# Project - Airline AI Assistant

We'll now bring together what we've learned to make an AI Customer Support assistant for an Airline

Airline ticket by using multiple tools

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

# As an alternative, if you'd like to use Ollama instead of OpenAI
# Check that Ollama is running for you locally (see week1/day2 exercise) then uncomment these next 2 lines
# MODEL = "llama3.2"
# openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')


OpenAI API Key exists and begins sk-proj-


In [3]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

## Tools


There  will be 2 tools at the beginnig. 
1: Will tell the ticket price
2. Will tell the places to see

In [4]:
# 1: Ticket price function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}") # this way we can see this function called in output
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown") # ticket_prices'te bulamazsa Unknown yazıyor
    # return ticket_prices.get(city) # bu boş dönüyor bulamazsa

In [5]:
# get_ticket_price("Berlin")

In [6]:
# There's a particular dictionary structure that's required to describe our function:
# 1st function for first tool

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price,\
                    for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [7]:
# 2: Places to see function

places_to_see = {"london": "London Eye", "paris": "Eifel", "tokyo": "Tokyo Drift", "berlin": "I don't know", \
                "istanbul": "Şükrü Saraçoğlu, Fenerbahçe Stadium", "bayburt": "Ortugu. \
                This place is where onur has born. Everybody should see there."}

def get_places_to_see(destination_city): 
    print(f"Tool get_places_to_see called for {destination_city}") # this way we can see this function called in output
    city = destination_city.lower()
    return places_to_see.get(city, "Bimiyorum ki valla Onur'a sor")

In [8]:
# get_places_to_see("Ankara")

In [9]:
# There's a particular dictionary structure that's required to describe our function:
# 2nd function for second tool

place_function = {
    "name": "get_places_to_see",
    "description": "Get the place to see in the destination city. \
                    Call this whenever you need to know the place to visit,\
                    for example when a customer asks 'Where can I visit in this city' or 'Where should I see in this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [10]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function}, 
        {"type": "function", "function": place_function}]

## Getting OpenAI to use our Tool


In [11]:
# def chat(message, history):
#     messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
#     response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
#     print(f"finish reason {response.choices[0].finish_reason}")
#     print(f"Response {response.choices[0].message.content}")

#     if response.choices[0].finish_reason=="tool_calls":
#         message = response.choices[0].message
#         print(f" \n, Tool call message: {message}, \n")
#         response, city = handle_tool_call(message)
#         messages.append(message)
#         messages.append(response)
#         response = openai.chat.completions.create(model=MODEL, messages=messages)
    
#     return response.choices[0].message.content

In [12]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    tool_call = message.tool_calls[0] # message already has which tool to call
    arguments = json.loads(tool_call.function.arguments) # because it is in json format
    city = arguments.get('destination_city')
    function_name = tool_call.function.name # this is just a string, look at message(call tool message)
    
    # price = get_ticket_price(city) we dont know which function to use
    func = 0
    if function_name == "get_ticket_price":
        price = get_ticket_price(city)
        func = 1
        tool_name = "ticket_price"
        
    elif function_name == "get_places_to_see":
        place = get_places_to_see(city)
        func = 2
        tool_name = "place_to_see"
        
    if func == 1:
        response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}), # destination_city tool call messagının içinde var
        "tool_call_id": tool_call.id
        }
    elif func == 2:
        response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"place": place}), # destination_city tool call messagının içinde var
        "tool_call_id": tool_call.id
        }
        
    return response, city , tool_name
    # i give tool_name s that if it is place_to_see it will draw the city

In [13]:
# gr.ChatInterface(fn=chat, type="messages").launch()

# # call id'de hata alıyor. şehir ismi verip taskı tam anlatmazsan karıştırıyor.

# Let's go multi-modal!!

We can use DALL-E-3, the image generation model behind GPT-4o, to make us some images

Let's put this in a function called artist.

### Price alert: each time I generate an image it costs about 4 cents - don't go crazy with images!

In [14]:
# Some imports for handling images

import base64
from io import BytesIO
from PIL import Image

In [15]:
# def artist(city):
#     image_response = openai.images.generate(
#             model="dall-e-2", # dall-e-3
#             prompt=f"An image representing a vacation in {city}, showing tourist spots and everything unique about {city}, in a vibrant pop-art style",
#             size="1024x1024",
#             n=1,
#             response_format="b64_json",
#         )
#     image_base64 = image_response.data[0].b64_json
#     image_data = base64.b64decode(image_base64)
#     return Image.open(BytesIO(image_data))

In [16]:
# image = artist("İstanbul")
# display(image)

In [17]:
# tool_call sonrası çizeceği şehir fotosuna place_to_see önerisini de dahil edecek.

def artist(city, artist_place_input):
    image_response = openai.images.generate(
            model="dall-e-3", # dall-e-3
            prompt=f"An image representing a vacation in {city}, showing tourist spots and \
            everything unique about {city}, in a vibrant pop-art style. Be creative and cool \
            Also if {artist_place_input} \
            is not None and there is a place suggested to visit, you must include that place \
            in the middle of theimage. For example if sugestion is to see Fenerbahçe Stadium, include \
            the stadium in the image. If it is Onur's hometown Ortugu then include Onur's home and village. \
            If there is no suggested place don't include it.",
            size="1024x1024",
            n=1,
            response_format="b64_json",
        )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

# Audio

In [18]:
!ffmpeg -version
!ffprobe -version
!ffplay -version

ffmpeg version 7.1.1 Copyright (c) 2000-2025 the FFmpeg developers
built with Apple clang version 16.0.0 (clang-1600.0.26.6)
configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1.1_2 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --e

In [19]:
from pydub import AudioSegment
from pydub.playback import play

def talker(message):
    response = openai.audio.speech.create(
      model="tts-1",
      voice="onyx",    # Also, try replacing onyx with alloy
      input=message
    )
    
    audio_stream = BytesIO(response.content)
    audio = AudioSegment.from_file(audio_stream, format="mp3")
    play(audio)

In [20]:
talker("Well, hi there")

Input #0, wav, from '/var/folders/mf/fgc19wdn3pnglmlpk_yq8m2w0000gn/T/tmpl299dcnu.wav':
  Duration: 00:00:00.84, bitrate: 384 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 24000 Hz, 1 channels, s16, 384 kb/s
   0.71 M-A:  0.000 fd=   0 aq=    0KB vq=    0KB sq=    0B 

# Our Agent Framework

The term 'Agentic AI' and Agentization is an umbrella term that refers to a number of techniques, such as:

1. Breaking a complex problem into smaller steps, with multiple LLMs carrying out specialized tasks
2. The ability for LLMs to use Tools to give them additional capabilities
3. The 'Agent Environment' which allows Agents to collaborate
4. An LLM can act as the Planner, dividing bigger tasks into smaller ones for the specialists
5. The concept of an Agent having autonomy / agency, beyond just responding to a prompt - such as Memory

We're showing 1 and 2 here, and to a lesser extent 3 and 5. In week 8 we will do the lot!

In [21]:
def chat(history):
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    image = None

    
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city, tool_name = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        # image = artist(city)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
        artist_place_input = response.choices[0].message.content
        
        if tool_name == "place_to_see":
            image = artist(city, artist_place_input)

        elif tool_name == "ticket_price":
            image = None

        artist_place_input = None

        
        
    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]

    # Audio
    talker(reply)
    
    return history, image

In [22]:
# More involved Gradio code as we're not using the preset Chat interface!
# Passing in inbrowser=True in the last line will cause a Gradio window to pop up immediately.

with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500)
    with gr.Row():
        entry = gr.Textbox(label="You can write here :D")
    with gr.Row():
        clear = gr.Button("Clear")

    def do_entry(message, history):
        history += [{"role":"user", "content":message}]
        return "", history

    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot, image_output]
    )
    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


Input #0, wav, from '/var/folders/mf/fgc19wdn3pnglmlpk_yq8m2w0000gn/T/tmpkxob3gov.wav':
  Duration: 00:00:01.99, bitrate: 384 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 24000 Hz, 1 channels, s16, 384 kb/s
   1.91 M-A:  0.000 fd=   0 aq=    0KB vq=    0KB sq=    0B 


Tool get_places_to_see called for Istanbul


Input #0, wav, from '/var/folders/mf/fgc19wdn3pnglmlpk_yq8m2w0000gn/T/tmpz8h7t2jn.wav':
  Duration: 00:00:02.90, bitrate: 384 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 24000 Hz, 1 channels, s16, 384 kb/s
   2.88 M-A:  0.000 fd=   0 aq=    0KB vq=    0KB sq=    0B 